In [213]:
%load_ext autoreload
%autoreload

import sys
sys.path.append('..')

import os
import pickle
import numpy as np
import pprint
import copy

import color.models.predict_color as predict_color_models
import color.utils.utils as utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def save_hp_config(hp_config, overwrite=False):
    hp_dir = os.path.join('..', 'trained_models', 'hp', hp_config['hp_name'])
    config_path = os.path.join(hp_dir, 'hp_config.pickle')
    os.makedirs(hp_dir, exist_ok=overwrite)
    with open(config_path, 'wb') as x:
        pickle.dump(hp_config, x)

In [3]:
# HP config template
hp_config = {
    'hp_name': None,
    'model_key': None,
    'model_params': [],
    'training_params': {
        'num_epochs': 10,
        'draw_plots': True,
        'show_progress': True,
        'do_cv': True,
        'use_cuda': True,
    },
    'dataset_params': {
        'dataset': 'big',
        'emb_len': 200,
        'normalize_rgb': True,
        'use_cuda': True,
        'pad_len': None,
        'batch_size': 1,
        'cv_split': 0.1,
        'test_split': 0,
        'num_workers': 0,
    }
}

In [4]:
# Test Hyperparams
hp_config_test = copy.deepcopy(hp_config)
hp_config_test['hp_name'] = 'hp_test'
hp_config_test['model_key'] = 'predict_color_rnn'
hp_config_test['model_params'] = [{
    'emb_dim': 50,
    'hidden_dim': 10,
    'num_layers': 1,
    'dropout': 0,
    'color_dim': 3,
    'nonlinearity': 'relu',
    'lr': 0.1,
    'momentum': 0.9,
    'weight_decay': 0
}]*2
if False:
    save_hp_config(hp_config_test)

In [220]:
# Generte Hyperparams for LSTM model (Color predictions)
num_models = 10
model_params = []
count = 0
while count < num_models:
    config = {
        'emb_dim': 200,
        'name': 'lstm_{}'.format("{}".format(count+1).zfill(3))
    }
    config['hidden_dim'] = int(np.random.choice([int(0.75*config['emb_dim']), config['emb_dim'],
                                            int(1.5*config['emb_dim']), int(2*config['emb_dim'])]))
    config['num_layers'] = int(np.random.choice([1,2,3,4]))
    
    config['dropout'] = -1
    while config['dropout'] < 0 or config['dropout'] > 0.4:
        config['dropout'] = float((0.2*np.random.randn()) + 0.2)
    
    config['lr'] = float(min(0.5, max(0.001, np.power(10, np.random.randn()-1.5))))
    config['momentum'] = float(min(0.97, max(0.8, np.random.normal(0.9, 0.8))))
    config['weight_decay'] = min(1e-04, float(np.power(10, np.random.randn()-4)))
    
    lr_step_size = int(np.random.choice(np.arange(1,11)))
    lr_gamma = float(max(0.95, 1 - np.power(10, -3*np.random.rand())))
    config['lr_decay'] = (lr_step_size, lr_gamma)
    
    model = predict_color_models.ColorPredictorLSTM(**config)
    num_params = utils.get_trainable_params(model)
    if num_params < 50_000 or num_params > 1_000_000:
        print('Num params out of range: {}. Skipping model:'.format(num_params))
        print(config)
        
    model_params.append(config)
    count += 1
    
for i, config in enumerate(model_params):
    print('Model', i+1)
    pprint.pprint(config)

Num params out of range: 1287003. Skipping model:
{'emb_dim': 200, 'name': 'lstm_003', 'hidden_dim': 200, 'num_layers': 4, 'dropout': 0.1939019411958394, 'lr': 0.0010568284723943514, 'momentum': 0.8, 'weight_decay': 3.709035542879697e-05, 'lr_decay': (6, 0.9986559254554298)}
Num params out of range: 1325703. Skipping model:
{'emb_dim': 200, 'name': 'lstm_005', 'hidden_dim': 300, 'num_layers': 2, 'dropout': 0.34224012399515436, 'lr': 0.04022831682749591, 'momentum': 0.8, 'weight_decay': 2.700974778463191e-07, 'lr_decay': (7, 0.9988013696464879)}
Num params out of range: 3530803. Skipping model:
{'emb_dim': 200, 'name': 'lstm_007', 'hidden_dim': 400, 'num_layers': 3, 'dropout': 0.3446285801932848, 'lr': 0.12201589765427247, 'momentum': 0.8920934509967821, 'weight_decay': 6.540268807490446e-05, 'lr_decay': (4, 0.95)}
Num params out of range: 2770503. Skipping model:
{'emb_dim': 200, 'name': 'lstm_009', 'hidden_dim': 300, 'num_layers': 4, 'dropout': 0.10071983955562538, 'lr': 0.01668169107

/home/rb/.virtualenvs/colornamer/lib/python3.7/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1370067640922164 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/home/rb/.virtualenvs/colornamer/lib/python3.7/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.13896589593248554 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/home/rb/.virtualenvs/colornamer/lib/python3.7/site-packages/torch/nn/modules/rnn.py:46: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.17859870944238176 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [228]:
# # Generte Hyperparams for LSTM model (Color predictions)
# num_models = 10
# model_params = []
# count = 0
# while count < num_models:
#     config = {
#         'emb_dim': 200
#     }
#     config['hidden_dim'] = int(np.random.choice([int(0.75*config['emb_dim']), config['emb_dim'],
#                                             int(1.5*config['emb_dim']), int(2*config['emb_dim'])]))
#     config['num_layers'] = int(np.random.choice([1,2,3,4]))
#     use_dropout = np.random.choice([0, 1]) if config['num_layers'] > 1 else 0
#     config['dropout'] = float(0.4*np.random.rand()) if use_dropout == 1 else 0
    
#     config['lr'] = float(np.power(10, -3.5 * np.random.rand()))
#     config['momentum'] = float(1 - 0.2*np.random.rand())
#     config['weight_decay'] = float(np.power(10, -4 * np.random.rand()))
    
#     model = predict_color_models.ColorPredictorLSTM(**config)
#     num_params = utils.get_trainable_params(model)
#     if num_params < 50_000 or num_params > 3_000_000:
#         print('Num params out of range: {}. Skipping model:'.format(num_params))
#         print(config)
        
#     model_params.append(config)
#     count += 1
    
# for i, config in enumerate(model_params):
#     print('Model', i+1)
#     pprint.pprint(config)

In [227]:
# Save Hyperparams for LSTM model (Color predictions)
hp_config_lstm = copy.deepcopy(hp_config)
hp_config_lstm['hp_name'] = 'predict_color_lstm_r1'
hp_config_lstm['model_key'] = 'predict_color_lstm'
hp_config_lstm['model_params'] = model_params
if True:
    save_hp_config(hp_config_lstm, overwrite=False)